In [1]:
import pandas as pd
import numpy as np
from __future__ import print_function
import re, gzip, time, itertools, io
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Bio
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
import json

In [7]:
df= pd.read_csv("HG38_FULL_IDR_filtered_wOmega.csv")

In [8]:
df.head()

,index,ID,PepID,Gene_symbol,IDR_START,IDR_END,IDRlengths,IDRseq,IDRfrac_aromatic,IDRNum_aromatic,bsize,omega
0,6,ENSP00000002501_IDR_1,ENSP00000002501,DBNDD1,0,158,158,MEPPEGAGTGEIVKEAEVPQAALGVPAQGTGDNGHTPVEEEVGGIP...,0.031646,5,32,0.202422
1,12,ENSP00000003084_IDR_2,ENSP00000003084,CFTR,644,837,193,MGCDSFDQFSAERRNSILTETLHRFSLEGDAPVSWTETKKQSFKQT...,0.051813,10,19,0.287500
2,25,ENSP00000005226_IDR_1,ENSP00000005226,USH1C,503,697,194,ADNEISEMTTGPPPPPPSVSPLAPPLRRFAGGLHLHTTDLDDIPLD...,0.046392,9,22,0.245973
3,27,ENSP00000005260_IDR_1,ENSP00000005260,BAIAP2L1,246,511,265,KTPASTPVSGTPQASPMIERSNVVRKDYDTLSKCSPKMPPAPSGRA...,0.052830,14,19,0.357020
4,29,ENSP00000005284_IDR_2,ENSP00000005284,CACNG3,218,315,97,STFARLPPYRYRFRRRSSSRSTEPRSRDLSPISKGFHTIPSTDISM...,0.092784,9,11,0.553339


In [9]:
with open('HG38_pep_unique_onlyids.fasta') as fasta_file:  # Will close handle cleanly
    identifiers = []
    for title, sequence in SimpleFastaParser(fasta_file):
        identifiers.append(title)
s1 = pd.Series(identifiers, name='PepID')
df2 = pd.DataFrame(dict(PepID=s1))
df2['PepID2'] = df2['PepID'].str.split('.').str[0]
df2.head()

,PepID,PepID2
0,ENSP00000399668.2,ENSP00000399668
1,ENSP00000302319.1,ENSP00000302319
2,ENSP00000250823.4,ENSP00000250823
3,ENSP00000342812.3,ENSP00000342812
4,ENSP00000303300.1,ENSP00000303300


In [10]:
df = pd.merge(df, df2, how="left", left_on="PepID", right_on="PepID2")
df.head()

,index,ID,PepID_x,Gene_symbol,IDR_START,IDR_END,IDRlengths,IDRseq,IDRfrac_aromatic,IDRNum_aromatic,bsize,omega,PepID_y,PepID2
0,6,ENSP00000002501_IDR_1,ENSP00000002501,DBNDD1,0,158,158,MEPPEGAGTGEIVKEAEVPQAALGVPAQGTGDNGHTPVEEEVGGIP...,0.031646,5,32,0.202422,ENSP00000002501.6,ENSP00000002501
1,12,ENSP00000003084_IDR_2,ENSP00000003084,CFTR,644,837,193,MGCDSFDQFSAERRNSILTETLHRFSLEGDAPVSWTETKKQSFKQT...,0.051813,10,19,0.287500,ENSP00000003084.6,ENSP00000003084
2,25,ENSP00000005226_IDR_1,ENSP00000005226,USH1C,503,697,194,ADNEISEMTTGPPPPPPSVSPLAPPLRRFAGGLHLHTTDLDDIPLD...,0.046392,9,22,0.245973,ENSP00000005226.7,ENSP00000005226
3,27,ENSP00000005260_IDR_1,ENSP00000005260,BAIAP2L1,246,511,265,KTPASTPVSGTPQASPMIERSNVVRKDYDTLSKCSPKMPPAPSGRA...,0.052830,14,19,0.357020,ENSP00000005260.8,ENSP00000005260
4,29,ENSP00000005284_IDR_2,ENSP00000005284,CACNG3,218,315,97,STFARLPPYRYRFRRRSSSRSTEPRSRDLSPISKGFHTIPSTDISM...,0.092784,9,11,0.553339,ENSP00000005284.4,ENSP00000005284


In [11]:
df=df[['PepID_y', 'IDR_START', 'IDR_END']]
df.rename(columns = {'PepID_y':'PepID',}, inplace = True)
df['index']=df['IDR_START'].astype(str)+" "+df['IDR_END'].astype(str)
df=df.drop(['IDR_START'], axis=1)
df=df.drop(['IDR_END'], axis=1)
df_new=df.groupby('PepID').apply(lambda dfg: dfg.drop('PepID', axis=1).to_dict(orient='list')).to_dict()

In [12]:
indexesforkeys={}

for i in df_new.keys():
    aa=(list((df_new[i]).values())[0])
    ab= [i.split(" ", 1) for i in aa]
    flat_list = [item for sublist in ab for item in sublist]
    indexesforkeys[i]=list(map(int, flat_list))

In [13]:
##save metapredict IDR indexes
with open("metapredict_concIDRs.json", "w") as fp:
    json.dump(indexesforkeys , fp) 

In [14]:
indexesforkeys

{'ENSP00000002501.6': [0, 158],
 'ENSP00000003084.6': [644, 837],
 'ENSP00000005226.7': [503, 697],
 'ENSP00000005260.8': [246, 511],
 'ENSP00000005284.4': [218, 315],
 'ENSP00000005340.4': [93, 258, 519, 736],
 'ENSP00000005386.3': [42, 134, 413, 552],
 'ENSP00000006015.3': [0, 247],
 'ENSP00000006053.6': [103, 333],
 'ENSP00000006526.6': [0, 68],
 'ENSP00000006777.6': [233, 364],
 'ENSP00000007414.3': [146, 475],
 'ENSP00000007699.5': [131, 364],
 'ENSP00000007735.3': [0, 55],
 'ENSP00000008391.4': [0, 182],
 'ENSP00000011619.3': [455, 604],
 'ENSP00000014914.6': [273, 357],
 'ENSP00000015877.6': [147, 233],
 'ENSP00000016913.4': [0, 76],
 'ENSP00000016946.3': [755, 1049, 1208, 1353, 1465, 1644],
 'ENSP00000017003.2': [0, 160],
 'ENSP00000020673.5': [0, 550],
 'ENSP00000020926.3': [0, 124],
 'ENSP00000020945.1': [0, 121],
 'ENSP00000025008.5': [638, 797],
 'ENSP00000025301.2': [0, 59, 103, 544, 709, 1334, 1373, 1816],
 'ENSP00000029410.5': [0, 82],
 'ENSP00000033079.3': [216, 569],
 